<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [1]:
### YOUR CODE STARTS HERE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel


import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context



In [2]:
names = ['class_name',
         'handicapped-infants',
         'water-project-cost-sharing',
         'adoption-of-the-budget-resolution',
         'physician-fee-freeze',
         'el-salvador-aid',
         'religious-groups-in-schools',
         'anti-satellite-test-ban',
         'aid-to-nicaraguan-contras',
         'mx-missile',
         'immigration',
         'synfuels-corporation-cutback',
         'education-spending',
         'superfund-right-to-sue',
         'crime',
         'duty-free-exports',
         'export-administration-act-south-africa']


df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', 
                 header=None, 
                 names=names,
                 na_values='?')

# Replace strings with numbers so we can use them for
# math later
df = df.replace({'n':0,
                 'y':1})



df.head()

,class_name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [3]:
# Subset republican and democrat
republicans = df[df['class_name'] == 'republican']
republicans = republicans.drop('class_name', axis = 1)

democrats = df[df['class_name'] == 'democrat']
democrats = democrats.drop('class_name', axis = 1)

In [5]:
democrats.shape


(267, 16)

In [5]:
republicans.shape

(168, 16)

In [6]:
republicans.shape

(168, 16)

In [22]:
# Obviously, the larger the difference between means
# ,the higher the tvalue is

,issue,r_support,r_mean,d_support,d_mean,t_value,p_value,mean_diff
1,water-project-cost-sharing,44.64,0.506757,44.94,0.502092,0.09,0.929156,0.004665


In [23]:
# Fidn the issues that democrats support more with a p < 0.01
ltd = lt[lt['d_support'] > lt['r_support']]
ltd.head()

,issue,r_support,r_mean,d_support,d_mean,t_value,p_value,mean_diff
0,handicapped-infants,18.45,0.187879,58.43,0.604651,9.21,1.613440e-18,0.416772
2,adoption-of-the-budget-resolution,13.10,0.134146,86.52,0.888462,23.21,2.070340e-77,0.754315
6,anti-satellite-test-ban,23.21,0.240741,74.91,0.772201,12.53,8.521033e-31,0.531460
7,aid-to-nicaraguan-contras,14.29,0.152866,81.65,0.828897,18.05,2.824718e-54,0.676031
8,mx-missile,11.31,0.115152,70.41,0.758065,16.44,5.030793e-47,0.642913


In [24]:
# Fidn the issues that republicans support more with a p < 0.01
ltr = lt[lt['r_support'] > lt['d_support']]
ltr.head()

,issue,r_support,r_mean,d_support,d_mean,t_value,p_value,mean_diff
3,physician-fee-freeze,97.02,0.987879,5.24,0.054054,49.37,1.994262e-177,0.933825
4,el-salvador-aid,93.45,0.951515,20.60,0.215686,21.14,5.600520e-68,0.735829
5,religious-groups-in-schools,88.69,0.897590,46.07,0.476744,9.74,2.393672e-20,0.420846
11,education-spending,80.36,0.870968,13.48,0.144578,20.50,1.883420e-64,0.726389
12,superfund-right-to-sue,80.95,0.860759,27.34,0.289683,13.51,1.227858e-34,0.571077


In [25]:
lt.head().sort_values(by='mean_diff')

,issue,r_support,r_mean,d_support,d_mean,t_value,p_value,mean_diff
0,handicapped-infants,18.45,0.187879,58.43,0.604651,9.21,1.613440e-18,0.416772
5,religious-groups-in-schools,88.69,0.897590,46.07,0.476744,9.74,2.393672e-20,0.420846
4,el-salvador-aid,93.45,0.951515,20.60,0.215686,21.14,5.600520e-68,0.735829
2,adoption-of-the-budget-resolution,13.10,0.134146,86.52,0.888462,23.21,2.070340e-77,0.754315
3,physician-fee-freeze,97.02,0.987879,5.24,0.054054,49.37,1.994262e-177,0.933825


In [26]:
# Add another feature that shows whether
# we reject or accept the null hypothesis
dftp['conclusion'] = ['fail to reject' if x >= 0.5 else 'reject' for x in dftp['p_value']]

In [27]:
dftp.head(20)

,issue,r_support,r_mean,d_support,d_mean,t_value,p_value,mean_diff,conclusion
0,handicapped-infants,18.45,0.187879,58.43,0.604651,9.21,1.613440e-18,0.416772,reject
1,water-project-cost-sharing,44.64,0.506757,44.94,0.502092,0.09,9.291557e-01,0.004665,fail to reject
2,adoption-of-the-budget-resolution,13.10,0.134146,86.52,0.888462,23.21,2.070340e-77,0.754315,reject
3,physician-fee-freeze,97.02,0.987879,5.24,0.054054,49.37,1.994262e-177,0.933825,reject
4,el-salvador-aid,93.45,0.951515,20.60,0.215686,21.14,5.600520e-68,0.735829,reject
5,religious-groups-in-schools,88.69,0.897590,46.07,0.476744,9.74,2.393672e-20,0.420846,reject
6,anti-satellite-test-ban,23.21,0.240741,74.91,0.772201,12.53,8.521033e-31,0.531460,reject
7,aid-to-nicaraguan-contras,14.29,0.152866,81.65,0.828897,18.05,2.824718e-54,0.676031,reject
8,mx-missile,11.31,0.115152,70.41,0.758065,16.44,5.030793e-47,0.642913,reject
9,immigration,54.76,0.557576,46.44,0.471483,1.74,8.330248e-02,0.086093,reject


In [ ]:
# The only issue in which we fail to reject
# the null hypothesis is "water-project-cost-sharing".
# The reason is, the difference is not significant 
# enough